# 202108090846_XGBoostRegressor_tree_sweep
Another sweep -- this time with some different hyperparameters, based on the [first sweep's results](https://wandb.ai/hushifang/202108_Kaggle_tabular_playground/sweeps/4abje3y9?workspace=user-hushifang). This one will be for tree models only (plus DART).

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold#, KFold
from sklearn.metrics import mean_squared_error
import wandb
from wandb.xgboost import wandb_callback
# import timm
from pathlib import Path
import os
import math

In [10]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torchinfo import summary
# # from fastai.callback.wandb import *
# from torch.cuda.amp import autocast, GradScaler

In [11]:
# from fastai.vision.all import *
# from fastai.tabular.all import *
# from fastai.callback.wandb import WandbCallback

In [12]:
# import xgboost as xgb
from xgboost import XGBRegressor, XGBRFRegressor
# from xgboost import XGBClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.tree import DecisionTreeClassifier

In [13]:
# %config Completer.use_jedi = False

In [14]:
# set_seed(42, reproducible=True)

In [15]:
datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/202108_august/')

In [16]:
df = pd.read_csv(datapath/'train.csv', index_col='id', low_memory=False)

In [17]:
# df.isnull().sum().any() # False

In [18]:
# df.info(verbose=True, null_counts=True)

So no NaNs.

In [19]:
# df.describe()

In [20]:
y = df.loss

In [21]:
features = [x for x in df.columns if x != 'loss']

In [22]:
X = df[features]

In [23]:
len(X)

250000

In [24]:
df.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,loss
id,,,,,,,,,,,,,,,,,,,,,
0,-0.002350,59,0.766739,-1.350460,42.2727,16.68570,30.3599,1.267300,0.392007,1.09101,...,-42.43990,26.854000,1.45751,0.696161,0.941764,1.828470,0.924090,2.29658,10.48980,15
1,0.784462,145,-0.463845,-0.530421,27324.9000,3.47545,160.4980,0.828007,3.735860,1.28138,...,-184.13200,7.901370,1.70644,-0.494699,-2.058300,0.819184,0.439152,2.36470,1.14383,3
2,0.317816,19,-0.432571,-0.382644,1383.2600,19.71290,31.1026,-0.515354,34.430800,1.24210,...,7.43721,37.218100,3.25339,0.337934,0.615037,2.216760,0.745268,1.69679,12.30550,6
3,0.210753,17,-0.616454,0.946362,-119.2530,4.08235,185.2570,1.383310,-47.521400,1.09130,...,9.66778,0.626942,1.49425,0.517513,-10.222100,2.627310,0.617270,1.45645,10.02880,2
4,0.439671,20,0.968126,-0.092546,74.3020,12.30650,72.1860,-0.233964,24.399100,1.10151,...,290.65700,15.604300,1.73557,-0.476668,1.390190,2.195740,0.826987,1.78485,7.07197,1


In [25]:
len(y), len(y.value_counts())

(250000, 43)

So, it seems that we're actually looking at a classification problem here -- not a regression one.

# XGBRegressor

For the Sweep, you have to encapsulate your configs, your `wandb.init`, and everything else into a single function that will be called on each sweep iteration.

In [26]:
sweep_config = {
    "method": "grid", # try grid or random
    "metric": {
      "name": "rmse",
      "goal": "minimize"   
    },
    "parameters": {
        "booster": {
            "values": ["gbtree", "dart"]
        },
        "max_depth": {
            "values": [3, 6, 9, 12]
        },
        "learning_rate": {
            "values": [0.1, 0.2, 0.3]
        },
        "subsample": {
            "values": [1, 0.5]
        },
        "n_estimators": {
            "values": [50, 125, 200]
        }
    }
}

In [27]:
# %env "WANDB_NOTEBOOK_NAME" "202108090846_XGBoostRegressor_tree_sweep"
os.environ['WANDB_NOTEBOOK_NAME'] = '202108090846_XGBoostRegressor_tree_sweep.ipynb'

In [28]:
sweep_id = wandb.sweep(sweep_config, project="202108_Kaggle_tabular_playground")

Create sweep with ID: 81jyype5
Sweep URL: https://wandb.ai/hushifang/202108_Kaggle_tabular_playground/sweeps/81jyype5


In [29]:
def train():
    config_defaults = {
        "library": "xgboost",
        "tree_method": "auto", # set to 'gpu_hist' to try GPU if available
        "booster": 'gbtree',
        "n_estimators": 100,
        "max_depth": 6,
        "learning_rate": 0.1,
        "subsample": 1,
        "seed": 42,
        "test_size": 0.2,
    }

    wandb.init(
#         project="202108_Kaggle_tabular_playground",
        save_code=True,
        tags=['XGBoost'],
        name='202108080949_XGBRegressor_tree_sweep',
        notes='XGBRegressor sweep using tree boosting (gbtree and dart) and the GPU',
        config=config_defaults)
    
    config = wandb.config
        
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=config.test_size, random_state=config.seed)
    
    model = XGBRegressor(
        tree_method=config.tree_method,
        booster=config.booster,
        n_estimators=config.n_estimators, 
        max_depth=config.max_depth,
        learning_rate=config.learning_rate, 
        subsample=config.subsample,
        random_state=config.seed,
        test_size=config.test_size,
        n_jobs=-1, 
        verbosity=1, 
    )
    wandb.log({'params': model.get_params()})
    model.fit(X_train, y_train, callbacks=[wandb.xgboost.wandb_callback()])
    y_preds = model.predict(X_valid)
    mse = mean_squared_error(y_valid, y_preds)
    rmse = math.sqrt(abs(mse))
    wandb.log({'mse':mse, 'rmse':rmse})
    print(f"MSE is {mse}\nRMSE is {rmse}")   
    

In [30]:
?wandb.agent

In [ ]:
wandb.agent('mwlm6rsr', function=train)  

wandb: Agent Starting Run: ahcbblr0 with config:
wandb: 	booster: dart
wandb: 	learning_rate: 0.1
wandb: 	max_depth: 12
wandb: 	n_estimators: 50
wandb: 	subsample: 1
wandb: Currently logged in as: hushifang (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.11.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/xgboost/training.py:17: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)
/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


MSE is 62.221654832952375
RMSE is 7.888070412525003


_runtime,56
_timestamp,1628618243
_step,51
mse,62.22165
rmse,7.88807


_runtime,▁█
_timestamp,▁█
_step,▁█
mse,▁
rmse,▁


...

MSE is 62.85211582424859
RMSE is 7.927932632423701


_runtime,46
_timestamp,1628618332
_step,51
mse,62.85212
rmse,7.92793


_runtime,▁█
_timestamp,▁█
_step,▁█
mse,▁
rmse,▁


wandb: Agent Starting Run: i9xs7nyr with config:
wandb: 	booster: dart
wandb: 	learning_rate: 0.1
wandb: 	max_depth: 12
wandb: 	n_estimators: 125
wandb: 	subsample: 1
wandb: wandb version 0.11.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/xgboost/training.py:17: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)


# Old:

In [31]:
model.fit(X_train, y_train, callbacks=[wandb.xgboost.wandb_callback()])
# model.fit(X, y)

/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/xgboost/training.py:17: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=200, n_jobs=-1, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=1)

In [32]:
# wandb.sklearn.plot_regressor(model, X_train, X_valid, y_train, y_valid,  model_name='RandomForestRegressor', )

In [33]:
# y_pred = model.predict(X_valid)

In [34]:
# from sklearn.model_selection import cross_val_score

In [35]:
# scores = cross_val_score(model, X, y, cv=5, n_jobs=-1)

In [36]:
# model.get_depth()

In [38]:
y_valid_preds = model.predict(X_valid)

/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [39]:
mse = mean_squared_error(y_valid, y_valid_preds)

In [40]:
mse

61.500556614829804

In [41]:
rmse = math.sqrt(mse)
rmse

7.842229059064126

In [ ]:
wandb.log({'rmse': rmse, 'mse': mse, 'y_valid_preds': y_valid_preds})

In [42]:
# rmses = [math.sqrt(abs(score)) for score in scores]
# rmses

Let's give that a submit...

In [43]:
test_df = pd.read_csv(datapath/'test.csv', index_col='id', low_memory=False)

In [44]:
test_df.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
id,,,,,,,,,,,,,,,,,,,,,
250000,0.812665,15,-1.239120,-0.893251,295.5770,15.87120,23.04360,0.942256,29.898000,1.11394,...,0.446389,-422.332,-1.44630,1.69075,1.059300,-3.010570,1.94664,0.529470,1.386950,8.78767
250001,0.190344,131,-0.501361,0.801921,64.8866,3.09703,344.80500,0.807194,38.421900,1.09695,...,0.377179,10352.200,21.06270,1.84351,0.251895,4.440570,1.90309,0.248534,0.863881,11.79390
250002,0.919671,19,-0.057382,0.901419,11961.2000,16.39650,273.24000,-0.003300,37.940000,1.15222,...,0.990140,3224.020,-2.25287,1.55100,-0.559157,17.838600,1.83385,0.931796,2.336870,9.05400
250003,0.860985,19,-0.549509,0.471799,7501.6000,2.80698,71.08170,0.792136,0.395235,1.20157,...,1.396880,9689.760,14.77150,1.41390,0.329272,0.802437,2.23251,0.893348,1.359470,4.84833
250004,0.313229,89,0.588509,0.167705,2931.2600,4.34986,1.57187,1.118300,7.754630,1.16807,...,0.862502,2693.350,44.18050,1.58020,-0.191021,26.253000,2.68238,0.361923,1.532800,3.70660


In [45]:
X_test = test_df[features] # this is just for naming consistency

In [46]:
y_test_preds = model.predict(X_test)

/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [47]:
sample_df = pd.read_csv(datapath/'sample_submission.csv')

In [48]:
sample_df.loc[:, 'loss'] = y_test_preds

In [52]:
sample_df.head()

,id,loss
0,250000,7.328520
1,250001,4.610067
2,250002,7.277142
3,250003,7.030011
4,250004,7.187167


In [50]:
sample_df.to_csv('202108062038_XGBoost.csv', index=False)

In [51]:
wandb.finish()